In [1]:
import torch

In [2]:
from tqdm.notebook import tqdm  #shows how far forloop is gone (visually)
import pandas as pd

In [3]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 10.7 MB/s 
     |████████████████████████████████| 596 kB 38.4 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 895 kB 49.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

In [5]:
from google.colab import files
uploaded = files.upload()

Saving applestore_data_cleaned.csv to applestore_data_cleaned.csv


In [6]:
import io
df = pd.read_csv(io.BytesIO(uploaded['applestore_data_cleaned.csv']))

In [8]:
df.head()

,Unnamed: 0,Unnamed: 0.1,id,app_name,app_desc,category,language,category1,app_desc_clean,app_name_clean,app_name_desc_clean,cat1
0,0,0,281656475,PAC-MAN Premium,"SAVE 20%, now only $3.99 for a limited time!\n...",Games,en,GAME,save limited time one popular video games arca...,pac man premium,pac man premium save limited time one popular ...,0
1,1,1,281796108,Evernote - stay organized,Let Evernote change the way you organize your ...,Productivity,en,APP,let evernote change way organize personal prof...,evernote organized,evernote organized let evernote change way org...,1
2,2,2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",Download the most popular free weather app pow...,Weather,en,APP,popular weather powered largest professional w...,weatherbug local weather radar maps alerts,weatherbug local weather radar maps alerts pop...,1
3,3,3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",The eBay app is the best way to find anything ...,Shopping,en,APP,ebay best way find anything need dresses fashi...,ebay best buy sell save shopping,ebay best buy sell save shopping ebay best way...,1
4,4,4,282935706,Bible,On more than 250 million devices around the wo...,Reference,en,APP,million devices around world people reading li...,bible,bible million devices around world people read...,1


In [10]:
test_df = pd.read_csv("testing_data.csv")

In [24]:
df["name_desc"] = df["app_name"] + " " + df["app_desc"]

X_train = df["name_desc"]
y_train = df["cat1"]

X_val = test_df["user_query"]
y_val = test_df["category_1"]


In [23]:
y_train.values

array(['fart sounds', 'funny face filter', 'emoji maker', 'fake gps',
       'minecraft cheats', 'watch movies', 'whip sound', 'netflix',
       'video player', 'karaoke', 'TED videos', 'learn french',
       'practice maths', 'brain teasers', 'learn alphabet',
       'educational games for kids', 'practice languages online',
       'learn body parts', 'driving test', 'camera', 'photo editor',
       'face swap', 'retouch photo', 'snapchat', 'collage maker',
       'video editor', 'video effects', 'adobe', 'speed test',
       'calculator', 'russian keyboard', 'compass', 'currency converter',
       'sound recorder', 'qr scanner', 'google docs', 'notes',
       'calendar app', 'email client', 'sync devices', 'take notes',
       'period tracker', 'sleep monitor', 'calorie counter', 'yoga',
       'running app', 'diet tracker', 'headspace', 'drink water reminder',
       'spotify', 'listen to songs online', 'car radio', 'download music',
       'music player', 'create mixtape', 'identif

In [55]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(  #can tale multiple strings and encode them as we need them
    X_train.values, 
    add_special_tokens=True, 
    return_attention_mask=True, #binary mask indicating where model should attend to [1,1,1,0,0] 0 = padded instances
    pad_to_max_length=True, 
    max_length=300, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=300, 
    return_tensors='pt'
)

#ENCODED VALS IS A DICTIONARY - need to access that dictionary and pull out what we need

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train) #standard way of using dataset in pytorch
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [26]:
len(dataset_train)

6110

In [27]:
dataset_train[102]  #all correct - the title, the attention mask and the label

(tensor([  101,  2003,  4213,  2361,  5344,  5507,  2063,  1000,  1996,  1001,
          1015,  2227, 19948,  4691, 10439,  2144,  2268,  1000,  2412,  2359,
          2000, 19948,  2048,  5344,  1999,  1037,  6302,  1029,  2085,  2017,
          2064,   999,  2009,  2109,  2000,  2202,  1037,  7760, 18471,  3040,
          2000, 22291,  2028,  2227,  3031,  2178,  1010,  2021,  2003,  4213,
          2361,  5344, 29656,  3084,  2009,  2061,  3733,  2000,  6942,  5344,
          2008,  3087,  2064,  2079,  2009,  1999,  2104,  1037,  3371,  1012,
         22822,  8458,  2028,  2711,  2046,  2178,  2005,  3071,  1005,  1055,
          4024,   999,  2202,  1037,  3861,  2007,  2115, 18059,  1013, 25249,
          2030,  7170,  1037,  6302,  2013,  2115,  3075,  1010,  2173,  1996,
         15806,  2058,  2048,  5344,  1010,  1998,  2003,  4213,  2361,  5344,
         29656,  2515,  1996,  2717,   999, 19948,  5344,  1999,  2115,  2767,
          1005,  1055,  9130,  1010, 16021, 23091,  

In [28]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [60]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 10   #set to more when more hardware available e.g. 100

dataloader_train = DataLoader(dataset_train,      #iterators to iterate through dataset in batches
                              sampler=RandomSampler(dataset_train), #sampling randomly from training set
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), #doesnt matter here so can be sequential
                                   batch_size=batch_size)  #validation set smaller so batch size could be bigger

In [61]:
len(dataloader_train)

611

In [31]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=2e-5, #2e-5 based on original bert paper
                  eps=1e-8)  #this is default
                  
epochs = 2  
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)  


In [49]:
from sklearn.metrics import f1_score

#preds = [0.9, 0.05, 0, 0.05, 0]
#but we want preds = [1, 0 ,0 ,0 ,0]

label_dict = {"APP":1, "GAME":0}

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted') #class distrbution taken into account hance weighted, ca use macro too

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print('Accuracy:', len(y_preds[y_preds==label])/len(y_true))

def return_flat_lists(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return preds_flat, labels_flat
    
    

In [33]:
import numpy as np

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda




```
# This is formatted as code
```

# Let's train

In [35]:
def evaluate(dataloader_val):

    model.eval()  #evaluation mode- no backpropagation, eval mode freezes all weights
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():       #no gradients 
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()  #in case using GPU you want to pull them into CPU to use numpy
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    


In [36]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0 #adding each batches loss to this

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()  #only keep gradients for recurrent NN
       
       
        batch = tuple(b.to(device) for b in batch) #only important for GPU
        
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs) #unpacks dictionary into inputs https://stackoverflow.com/questions/36901/what-does-double-star-asterisk-and-star-asterisk-do-for-parameters
        
        loss = outputs[0]   #outputs is loss and logits, logits = outputs[1]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #prevent exploding or vanishing gradients

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/611 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.19507651679399202
Validation loss: 0.382900433712884
F1 Score (Weighted): 0.8195641809462378


Epoch 2:   0%|          | 0/611 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.09707029914230401
Validation loss: 0.29693377214042765
F1 Score (Weighted): 0.8735688010421284


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


note: loss is NOT a percentage 
https://stackoverflow.com/questions/34518656/how-to-interpret-loss-and-accuracy-for-a-machine-learning-model

In [66]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('finetuned_BERT_epoch_2.model', map_location=torch.device('cpu')))
print("model loaded")

_, predictions, true_vals = evaluate(dataloader_validation) # preferably have a test set
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

model loaded
Class: GAME
Accuracy: 0.875
Class: APP
Accuracy: 0.8715596330275229


In [67]:
f1_score_func(predictions, true_vals)

0.8735688010421284

In [68]:
preds, true = return_flat_lists(predictions, true_vals)

In [69]:
test_df["pred"] = preds

In [70]:
test_df[test_df.category_1 != test_df.pred]

,user_query,category_2,category_1,pred
1,funny face filter,Entertainment,1,0
4,minecraft cheats,Entertainment,1,0
13,brain teasers,Education,1,0
21,face swap,Photo & Video,1,0
31,compass,Utilities,1,0
36,notes,Productivity,1,0
53,music player,Music,1,0
68,premier league,Sports,1,0
75,buy clothes,Shopping,1,0
77,buy underwear,Shopping,1,0


In [44]:
#test = ["paris marathon", "cool adventure game", "word to pdf converter", "video call", "filter", "dating app", "note taking", "facebook"]
test = ["whip sound", "hangman", "brain teasers", "practice languages", "driving test", "speed test", "chat", "buy clothes"]
test_np = np.array(test)

test_y = np.array([1,0,1,1,1,1,1,1])

encoded_data_val = tokenizer.batch_encode_plus(
    test_np, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=5, 
    return_tensors='pt'
)

#ENCODED VALS IS A DICTIONARY - need to access that dictionary and pull out what we need

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(test_y)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train) #standard way of using dataset in pytorch
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), #doesnt matter here so can be sequential
                                   batch_size=batch_size)

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Class: GAME
Accuracy: 1.0
Class: APP
Accuracy: 0.7142857142857143


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
